In [1]:
import pandas as pd
from langdetect import detect
import string
import data_collector
import parser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pickle
import math
import numpy as np
import heapq
import re

# 1. Data Collection

## Get the list of the books
We already have the list of books in the pc, so we won't do it again.

Set to `True` both dirs, bests and links parameters to create the correct directories and download the txt containing all the html links.

In [2]:
data_collector.download_books(dirs=False, bests=False, links=False)

## 1.1 Crawl books
We already have all the htmls in the pc, so we won't do it again.

Set to `True` both the books and fails parameters to download all the html pages and remove the ones with broken pages.

In [3]:
data_collector.download_books(books=False, fails=False)

## 1.2 Parse downloaded pages
Set to `True` the create parameter to parse the downloaded html pages and create the tsv file.

In [4]:
parser.create_tsv(create=False)

In [2]:
df = pd.read_csv('parsed_books.tsv', sep='\t')

In [155]:
df.shape

(29959, 12)

In [156]:
df.head()

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,6408798.0,172554.0,"Could you survive on your own in the wild, wit...",374.0,September 14th 2008,Katniss Everdeen Peeta Mellark Cato (Hunger Ga...,"District 12, Panem Capitol, Panem Panem",https://www.goodreads.com/book/show/2767052-th...
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,J.K. Rowling,4.50,2525157.0,42734.0,There is a door at the end of a silent corrido...,870.0,September 2004,Sirius Black Draco Malfoy Ron Weasley Petunia ...,Hogwarts School of Witchcraft and Wizardry Lon...,https://www.goodreads.com/book/show/2.Harry_Po...
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,4527405.0,91802.0,The unforgettable novel of a childhood in a sl...,324.0,May 23rd 2006,Scout Finch Atticus Finch Jem Finch Arthur Rad...,"Maycomb, Alabama",https://www.goodreads.com/book/show/2657.To_Ki...
3,Pride and Prejudice,NaN,Jane Austen,4.26,3017830.0,67811.0,Alternate cover edition of ISBN 9780679783268S...,279.0,October 10th 2000,Mr. Bennet Mrs. Bennet Jane Bennet Elizabeth B...,"United Kingdom Derbyshire, England England Her...",https://www.goodreads.com/book/show/1885.Pride...
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,4989910.0,104912.0,About three things I was absolutely positive.F...,501.0,September 6th 2006,Edward Cullen Jacob Black Laurent Renee Bella ...,"Forks, Washington Phoenix, Arizona Washington ...",https://www.goodreads.com/book/show/41865.Twil...


## 1.3 Dataset cleaning [preliminary steps]
Before actually jumping into the work itself, we want our dataframe to be clean, meaning that there are some preliminary steps we need to perform on it. First of all, missing data is something we should pay attention to. Lot's of rows are going to have missing data somewhere, and dealing with missing data it's not that nice. Notice that this will include different strategies for each of the column we will be considering (more details below). Then there is the problem with punctuation, stopwords, stems and so on so forth, so basic text data preprocessing. Let's make a brief recap:

1. **Missing data**
    - `bookTitle`: if a book is missing the title, then we can safely just remove the instance. In fact, books that are missing the title are actually missing all the informations, meaning that there is a problem with the GoodReads specific link. Also, even if a book was missing just the title, we wouldn't have a way to refer to it, thus it wouldn't be really useful considering we're building a search engine.
    - `bookSeries`, `Authors`, `Plot`, `PublishingDate`, `Characters`, `Setting`: if a book is missing one of the above mentioned columns, we can still include the book in the data, since the search engine could for example work with just the title. Obviously, we cannot just leave the values missing, since it would be really hard to perform any operation on that. These are all text columns, therefore the best way to address the missing values prolem is to replace NaNs with empty strings.
    - `ratingValue`, `NumberofPages`: TODO?
2. **Text data preprocessing**
    - Punctuation removal: this is the first step we want to perform, since it is going to make the next steps much easier (e.g., language detection will be easier if there aren't plots composed just by punctuation symbols).
    - Language detection: before doing anything else, we want to remove the books that present the books for which the plot isn't in english.
    - Stopwords removal (of the `Plot` column only)
    - Stemming (of the `Plot` column only)
    - Lowercase

### Missing values

#### Title
There are 774 books that are completely empty, and these corresponds to the ones that are missing the `bookTitle` column. If you give a look at the url, you can see that these are not given by our python script to download and parse the books, but actually from the fact that the link is broken. Also, you can see that all the books that are missing the `bookTitle` are also missing all the remaining data.

This means that we can safely just remove all the rows that are missing the `bookTitle` column.

In [3]:
n_missing = df[(df['bookTitle'].isna())].shape[0]
print('There are {} instances that are missing the `bookTitle` column.'.format(n_missing))
print()
df[(df['bookTitle'].isna())].head()

There are 774 instances that are missing the `bookTitle` column.



,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/40937505\r\n
370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/30528535\r\n
379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/30528544\r\n
789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/40941582\r\n
1141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/5295735\r\n


In [4]:
# Remove empty books
df = df[(df['bookTitle'].notna())]

#### Text data

In [5]:
str_columns = ['bookSeries', 'bookAuthors', 'Plot', 'PublishingDate', 'Characters', 'Setting']

for col in str_columns:
    df[col] = df[col].fillna('')

### Text data preprocessing

#### Punctuation removal

**Observations**:

There are several ways to remove punctuations, including the use of exernal libraries (like nltk). But actually the fastest way to perform punctuation removal is the use of the internal methong translate, which is programmed in C and therefore it's much faster than the other options (give a look to this [link](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string) for a nice performance analysis of the various options).

In [9]:
s = 'This is a test string, asdw.ith lots of: punctuations; in it?!.'

In [12]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation + '’—'))

def remove_punctuation_(s):
    return re.sub("[^\w\s]", " ", s)

In [161]:
for col in str_columns:
    if col == 'Plot':
        df[col] = df[col].apply(remove_punctuation_)
    else:
        df[col] = df[col].apply(remove_punctuation)

#### Language detection
There are four possibilities `Plot` column of a given book:
1. It is written in english
2. It is written in another language
3. It is empty
4. It contains symbols, numbers, and so on

We want to keep only the ones written in english or empty, so we are just going to discard the others.

In [162]:
def language(s):
    if s == '':
        return 'empty'
    try:
        return detect(s)
    except:
        return 'symbols'

In [163]:
df['plot_lang'] = df['Plot'].apply(language)

In [164]:
df = df[df['plot_lang'] == 'en'].drop(columns=['plot_lang'])

In [165]:
df.shape

(26126, 12)

#### Stopwords removal
We are not going to perform stopwords removal on all the columns, since we could remove important things (e.g., we don't want to remove anything from the names of the characters). The only column on which stopwords removal is necessary is `Plot`.

In [166]:
def remove_stopwords(s):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(s)
    return ' '.join([w for w in tokens if w not in stop_words])

In [167]:
df['Plot'] = df['Plot'].apply(remove_stopwords)

#### Stemming
As for the stopwords removal, the only column on which stemming is necessary is `Plot`.

In [168]:
def stemming(s):
    ps = PorterStemmer()
    tokens = word_tokenize(s)
    return ' '.join([ps.stem(w) for w in tokens])

In [169]:
df['Plot'] = df['Plot'].apply(stemming)
df['Plot'] = df['Plot'].apply(stemming)

#### Lowercase
On the other hand, we want all the string columns to be lowercase, so that our search engine won't have problems with upper/lower case differences.

In [170]:
for col in str_columns:
    df[col] = df[col].apply(lambda w: w.lower())

In [171]:
df.head()

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
0,The Hunger Games,the hunger games 1,suzanne collins,4.33,6408798.0,172554.0,could surviv wild everi one make sure live see...,374.0,september 14th 2008,katniss everdeen peeta mellark cato hunger gam...,district 12 panem capitol panem panem,https://www.goodreads.com/book/show/2767052-th...
1,Harry Potter and the Order of the Phoenix,harry potter 5,jk rowling,4.50,2525157.0,42734.0,there door end silent corridor and haunt harri...,870.0,september 2004,sirius black draco malfoy ron weasley petunia ...,hogwarts school of witchcraft and wizardry lon...,https://www.goodreads.com/book/show/2.Harry_Po...
2,To Kill a Mockingbird,to kill a mockingbird,harper lee,4.28,4527405.0,91802.0,the unforgett novel childhood sleepi southern ...,324.0,may 23rd 2006,scout finch atticus finch jem finch arthur rad...,maycomb alabama,https://www.goodreads.com/book/show/2657.To_Ki...
3,Pride and Prejudice,,jane austen,4.26,3017830.0,67811.0,altern cover edit isbn 9780679783268sinc immed...,279.0,october 10th 2000,mr bennet mrs bennet jane bennet elizabeth ben...,united kingdom derbyshire england england hert...,https://www.goodreads.com/book/show/1885.Pride...
4,Twilight,the twilight saga 1,stephenie meyer,3.60,4989910.0,104912.0,about three thing i absolut posit first edward...,501.0,september 6th 2006,edward cullen jacob black laurent renee bella ...,forks washington phoenix arizona washington state,https://www.goodreads.com/book/show/41865.Twil...


In [172]:
df = df.reset_index(drop=True).reset_index()

### Save data

In [173]:
df.to_csv('clean_data.csv', index=False)

# 2. Search Engine

## 2.1 Conjunctive query

### 2.1.1 Create your index!

In [14]:
df = pd.read_csv('clean_data.csv')

In [15]:
# To save and load python dictionaries

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
def term_index(documents):
    words = set()
    for s in documents:
        try:
            tokens = set(word_tokenize(s))
            words.update(tokens)
        except:
            continue
        
    term_index = {}
    for i, word in enumerate(words):
        term_index[word] = i
    return term_index

In [17]:
term_indexes = term_index(df['Plot'])

In [18]:
save_obj(term_indexes, 'term_index')

In [19]:
def inverted_index(documents, term_indexes):
    inv_index = defaultdict(list)
    for i, s in enumerate(documents):
        try:
            tokens = set(word_tokenize(s))
            for token in tokens:
                token_index = term_indexes[token]
                inv_index[token_index].append(i)
        except:
            continue
    return inv_index

In [20]:
inv_indexes = inverted_index(df['Plot'], term_indexes)

In [21]:
save_obj(inv_indexes, 'inverted_index')

### 2.1.2 Execute the query

In [22]:
# Write it as a classs

class SimpleSearchEngine:
    def __init__(self, df, term_indexes, inv_indexes):
        self.df = df
        self.term_indexes = term_indexes
        self.inv_indexes = inv_indexes
        
    def execute_query(self, query):
        # Since we performed stemming on the plot column of the dataframe, we need to
        # perform stemming also on the query. Otherwise, our results wouldn't be accurate
        ps = PorterStemmer()
        query_tokens = set([ps.stem(w) for w in word_tokenize(query)])

        # Create term indexes for the query
        # notice: if one of the query element doesn't appear in the term_indexes dictionary
        # we can safely say that the **conjunctive** query has to return nothing
        term_indexes_tokens = []
        for token in query_tokens:
            if token in self.term_indexes.keys():
                term_indexes_tokens.append(self.term_indexes[token])
            else:
                print('No results')
                return pd.DataFrame(columns=['bookTitle', 'Plot', 'Url'])

        query_inv_indexes = {}
        for token_index in term_indexes_tokens:
            query_inv_indexes[token_index] = set(self.inv_indexes[token_index])

        # Since it is a conjuntive query, we need to intersect the results of each query token
        documents_id = sorted(set.intersection(*query_inv_indexes.values()))

        return self.df[self.df['index'].isin(documents_id)][['bookTitle', 'Plot', 'Url']]

In [23]:
simple_se = SimpleSearchEngine(df, term_indexes, inv_indexes)

In [25]:
simple_se.execute_query('survival games')

,bookTitle,Plot,Url
0,The Hunger Games,could surviv wild everi one make sure live see...,https://www.goodreads.com/book/show/2767052-th...
221,Catching Fire,spark are ignit flame are spread and the capit...,https://www.goodreads.com/book/show/6148028-ca...
319,Mockingjay,the final book ground break hunger game trilog...,https://www.goodreads.com/book/show/7260188-mo...
337,Legend,what western unit state home republ nation per...,https://www.goodreads.com/book/show/9275658-le...
652,The Magus,thi dare literari thriller rich erot suspen on...,https://www.goodreads.com/book/show/16286.The_...
...,...,...,...
25067,The Manhattan Hunt Club,in manhattan hunt club john saul plumb depth m...,https://www.goodreads.com/book/show/6553.The_M...
25531,Love's Forbidden Flower,plea note thi new adult romanc novel involv tw...,https://www.goodreads.com/book/show/16189423-l...
25806,The Southpaw,the southpaw stori come age america way baseb ...,https://www.goodreads.com/book/show/413736.The...
25827,Devil's Own,after surviv slaveri aiden macalpin noth thoug...,https://www.goodreads.com/book/show/8705483-de...


## 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index

In [26]:
def tfidf_inv_indexes(documents, term_indexes, inv_indexes, corpus=df):
    tfidf_indexes = defaultdict(dict)
    for doc_id, s in enumerate(documents):
        try:
            tokens = word_tokenize(s)
            tokens_set = set(tokens)
            n_tokens = len(tokens)
            norm = 0
            for token in tokens_set:
                token_index = term_indexes[token]
                # tf = n_times token appears in the document over the number of words of the document
                tf = s.count(token) / n_tokens
                # idf = log of the number of documents in the corpus over the number of documents in which the token appears
                idf = math.log10(len(corpus) / (len(inv_indexes[token_index])))
                tf_idf = tf * idf
                # we just computed the tfidf for the a particular token, for the document we're considering
                tfidf_indexes[token_index][doc_id] = tf * idf
                
                # Store also the norm for the document we're considering
                # which is sqrt of the sum of the squares
                norm += tf_idf ** 2
                
            # apply sqrt
            norm = np.sqrt(norm)
            # Normalize each document tfidf
            for token in tokens_set:
                token_index = term_indexes[token]
                tfidf_indexes[token_index][doc_id] /= norm
        except:
            continue
    return tfidf_indexes

In [27]:
tfidf_indexes = tfidf_inv_indexes(df['Plot'], term_indexes, inv_indexes)

In [28]:
save_obj(tfidf_indexes, 'tfidf_index')

### 2.2.2 Execute the query

In [38]:
class TfidfSearchEngine:
    def __init__(self, df, term_indexes, inv_indexes, tfidf_indexes, simple_se):
        self.df = df
        self.term_indexes = term_indexes
        self.inv_indexes = inv_indexes
        self.tfidf_indexes = tfidf_indexes
        self.simple_se = simple_se
        
    def execute_query(self, query, k):
        # First stem the query
        ps = PorterStemmer()
        query_tokens = set([ps.stem(w) for w in word_tokenize(query)])
        
        # Extract the token indexes from the vocabulary
        tokens_ids = []
        for token in query_tokens:
            try:
                tokens_ids.append(self.term_indexes[token])
            except:
                return
        
        # Compute the simple conjunctive query to get the books in which the query appears
        conj_query = self.simple_se.execute_query(query)
        
        # Extract the documents id for these books
        conj_query_ids = conj_query.index

        # Compute the similiarity
        scores = np.array([])
        for doc in conj_query_ids:
            tfidf = 0
            for token_id in tokens_ids:
                tfidf += self.tfidf_indexes[token_id][doc]
            
            scores = np.append(scores, tfidf)
        
        # Not really necessary since we do it for each book
        # scores /= np.sqrt(len(query.split()))
        
        conj_query['Similarity'] = scores

        # Use heaps to extract top k rows
        conj_query_list = conj_query.values.tolist()
        heapq.heapify(conj_query_list)
        max_k = heapq.nlargest(k, conj_query_list, key = lambda t: t[3])
        
        # Convert back to dataframe to show it
        max_k_df = pd.DataFrame(data=max_k, columns=['bookTitle', 'Plot', 'Url', 'Similarity'])

        return max_k_df

In [36]:
tfidf_se = TfidfSearchEngine(df, term_indexes, inv_indexes, tfidf_indexes, simple_se)

In [37]:
tfidf_se.execute_query('survival games', 1000)

,bookTitle,Plot,Url,Similarity
0,The Warden,alic led normal life she wake find trap sick g...,https://www.goodreads.com/book/show/33655366-t...,0.495827
1,Devil's Own,after surviv slaveri aiden macalpin noth thoug...,https://www.goodreads.com/book/show/8705483-de...,0.392048
2,The Quillan Games,let the game begin quillan territori verg dest...,https://www.goodreads.com/book/show/215540.The...,0.353052
3,The Hunger Games,could surviv wild everi one make sure live see...,https://www.goodreads.com/book/show/2767052-th...,0.318387
4,Truth,from new york time usa today bestsel author al...,https://www.goodreads.com/book/show/16070018-t...,0.247382
...,...,...,...,...
108,The Amber Room,the amber room one greatest treasur ever made ...,https://www.goodreads.com/book/show/5369.The_A...,0.024363
109,Hideaway,devil s night return hide place chase game bac...,https://www.goodreads.com/book/show/29082755-h...,0.024275
110,Fire & Blood,with fire furi fan come expect intern bestsel ...,https://www.goodreads.com/book/show/39943621-f...,0.022846
111,Dark Stranger: The Dream,smart spici the children the god paranorm roma...,https://www.goodreads.com/book/show/26005898-d...,0.018306
